YOU NEED TO CHANGE THE REFINEMENT PATCH TO ACCOMMODATE THE ORDER OF INTERPOLATION FOR COARSER GRIDS!!!

In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import csv
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML


sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST
from Modules import WaveformTools as WFT

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

In [2]:
nh_min = 16
refRatio = 2
CFL = np.sqrt(0.5)
x_0 = 0.
x_1 = 0.5
Hans = False

deriv = 'UD'
RK = 1

L = 1
locs = [x_1]
epsilons = [1, 1] # [1, 1]
mus = [1, 1] # [1, 1]

func = WFT.Gauss

dataPts = 5

AMR = True
plot = False

In [3]:
omega0 = BT.Grid(nh_min)
physics = BT.PhysProps(omega0, epsilons, mus, locs, L)

In [4]:
if (func == WFT.Polynomial):
    polyorder = 1
    x_0 = 0
    coefs = 16 * np.ones(polyorder+1, float)
    coefs[0] = 0
    args = [x_0, coefs]
else:
    sigma, mu = WFT.GaussParams(x_0, x_1)
    args = [sigma, mu]

In [5]:
def WaveEqConvergenceTest(omega0, physics0, CFL, RK, order, diff, func, args, dataPts, AMR, sameTime = False, numRefCel = 0, plot = True):
    override = True
    
    nh_min = omega0.nh_min
    epsilons = physics0.epsilons_r
    mus = physics0.mus_r
    locs = physics0.locs
    L = physics0.L
    cMat0 = physics0.cMat
    
    nt = 1
    if (sameTime):
        t0, nt = ST.CalcTime(omega0, CFL, cMat0, nt = nt)
    errorE = np.zeros(dataPts, float)
    errorB = np.zeros(dataPts, float)
    errorTot = np.zeros(dataPts, float)
    nhPower = np.arange(dataPts)
    nhVec = nh_min * (2 ** nhPower)
    refRatio = 2
    if (numRefCel == 0):
        halfO = np.ceil(order / 2)
    else:
        halfO = numRefCel
    print(nhVec)

    labels = ['Initial Wave', 'Exact Propagated Wave', 'RK-Propagated Wave']
    for i in range(dataPts):

        print('iter:', i)
        nh = 2 ** (i + 4)
        print('nh:', nh)
        omega = BT.Grid(nh)
        if (AMR):
            finepart = list(int(nh / 2) + np.arange(-int((halfO * nh) / nh_min), int((halfO * nh) / nh_min)))
            omega.AddPatch(refRatio, finepart)
            print(finepart)

        xCell = omega.xCell
        physics = BT.PhysProps(omega, epsilons, mus, locs, L)
        degFreed = omega.degFreed
        waves = WT.MakeWaves(omega)
        nullspace = OT.FindNullspace(omega, waves)
#         waves = waves @ nullspace
        wavesMat = OT.Block(waves, var = 2)
        nullspaceBlock = OT.Block(nullspace, var = 2)
        
        c = physics.cVec
        cMat = physics.cMat
        c1 = c[0]
        c2 = c[-1]

        if (sameTime):
            t, nt = ST.CalcTime(omega, CFL, cMat, t = t0)
            t = t0
        else:
            t, nt = ST.CalcTime(omega, CFL, cMat, nt = nt, override = override)
        

        waveInit = WFT.InitCond(omega, physics, func, args)
        FCoefs = FFTT.FourierCoefs(waves @ nullspace, waveInit)
        print('t:', t)

        wavePropEx = WFT.WaveEq(omega, physics, func, args, t)

        # Find propagated coeficients on coarse and fine grid.
        FCoefsProp = FFTT.FourierCoefs(waves @ nullspace, wavePropEx)

        solverCoefs = ST.RungeKutta(omega, physics, waveInit, CFL, nt, RK, order, deriv, ST.WaveEqRHS, override)
        wavePropSolv = wavesMat @ nullspaceBlock @ solverCoefs
        
        errorVec = wavePropEx - wavePropSolv
        errorTot[i] = TT.VectorNorm(abs(errorVec), 'max')
        errorE[i] = TT.VectorNorm(abs(errorVec[:nh])[order:-order], 'max')
        errorB[i] = TT.VectorNorm(abs(errorVec[nh:]), 'max')

        allCoefs = PT.Load(FCoefs, FCoefsProp, solverCoefs)
        allCoefs = nullspaceBlock @ allCoefs
        
        np.set_printoptions(precision=24, suppress=True)
        if ((locs != []) and (c1 == c2)):
            physics2 = BT.PhysProps(omega, [epsilons[0]], [mus[0]], [], L)
            solverCoefsAdvect = ST.RungeKutta(omega, physics2, waveInit, CFL, nt, RK, order, deriv, ST.AdvectRHS, override)
            wavePropExAdvectE = WFT.Advect(omega, physics2, func, args, t)
            wavePropExAdvectB = wavePropExAdvectE / c1
            wavePropExAdvect = np.append(wavePropExAdvectE, wavePropExAdvectB)
            wavePropSolvAdvect = wavesMat @ nullspaceBlock @ solverCoefsAdvect
            errorVecAdvect = wavePropExAdvect - wavePropSolvAdvect
            errorEAdvect = TT.VectorNorm(abs(errorVecAdvect[:nh])[order:-order], 'max')
            print('Max norm errors vs difference:', errorE[i], errorEAdvect, errorE[i] - errorEAdvect)
            if not all(wavePropEx == wavePropExAdvect):
                print('Exact Solution:')
                print(wavePropEx)
                print(wavePropExAdvect)
                print(wavePropEx - wavePropExAdvect)
                print('')
#                 sys.exit('Problem with Exact Solution!')
            if not all(errorVec[order:-order] == errorVecAdvect[order:-order]):
                print('Error:')
                print(errorVec)
                print(errorVecAdvect)
                print(errorVec - errorVecAdvect)
                print('')
#                 sys.exit('Problem with Error!')
            if not all(wavePropSolv[order:-order] == wavePropSolvAdvect[order:-order]):
                print('Solver Solution:')
                print(wavePropSolv)
                print(wavePropSolvAdvect)
                print(wavePropSolv - wavePropSolvAdvect)
                print('')
#                 sys.exit('Problem with Solver Solution!')
            if not all(solverCoefs[order:-order] == solverCoefsAdvect[order:-order]):
                print('Fourier Coefficients:')
                print(solverCoefs)
                print(solverCoefsAdvect)
                print(solverCoefs - solverCoefsAdvect)
                print('')
#                 sys.exit('Problem with Solver Solution Fourier Coefficients!')
        

        
        
        
        errorCoefs = PT.Load(FFTT.FourierCoefs(waves @ nullspace, errorVec))
        if (order == 1):
            orderString = str(order) + 'st'
        else:
            if (order == 2):
                orderString = str(order) + 'nd'
            else:
                if (order == 3):
                    orderString = str(order) + 'rd'
                else:
                    orderString = str(order) + 'th'
        
#         if (AMR and (RK == 4) and ((diff == 'U') or (diff == 'UD'))):
#             print('Such is the case.')
        if (plot and (order == 4)):
            PT.PlotMixedWave(omega, physics, waves = waves, FCoefs = allCoefs, rescale = 5, yGrid = True, labels = labels, plotCont = False, saveName = 'AMRPolyProp' + str(i))
#         if ((order == 4) or (order == 3)):
#             print('A plot was made.')
            PT.PlotMixedWave(omega, physics, waves = waves, FCoefs = nullspace @ errorCoefs[:degFreed], rescale = 5, yGrid = True, plotCont = False, saveName = 'CDOrder' + str(order)+ 'RK4Error' + str(i), title = 'Error on ' + orderString + '-Order Center RK4')
        header = ['x', 'anaytic', 'RK', 'error']

        savePath = '/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/Figures/'
        if (AMR):
            grid = 'AMR'
        else:
            grid = 'Uniform'

        data = [[x, y, z, a] for (x, y, z, a) in zip(xCell, wavePropEx[:degFreed], wavePropSolv[:degFreed], errorVec[:degFreed])]

        with open(savePath + grid + 'Order' + str(order) + 'CDRK4degfreed' + str(nh) + '.csv', 'w', encoding='UTF8', newline='') as f:
            writer = csv.writer(f)

            # write the header
            writer.writerow(header)

            # write the data
            writer.writerows(data)
#         print('')

    print('DONE')
    print('')
    print('')
    return nhVec, errorE
    

In [6]:
if ((deriv == 'U') or (deriv == 'UD')):
    faceApprox = 'Upwind'
    orders = 3
else:
    if ((deriv == 'C') or (deriv == 'CD')):
        faceApprox = 'Center'
        orders = 3
    else:
        sys.exit('Invalid entry for deriv!')

if (AMR):
    grid = 'AMR'
else:
    grid = 'Uniform'

if ((locs == []) or ((epsilons[0] == epsilons[1]) and (mus[0] == mus[1]))):
    meds = 'SingleMed'
else:
    meds = 'DualMed'

savePath = '/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/Figures/'
saveName = deriv + 'RK' + str(RK)
saveStringLog = savePath + saveName + grid + 'log'
saveStringLin = savePath + saveName + grid + 'line'


In [7]:
header = ['x', 'error']
xVals = list(np.arange(10))
yVals = list(6*np.arange(10))
data = [[x, y] for (x, y) in zip(xVals, yVals)]

with open(savePath + 'test.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write the data
    writer.writerows(data)

In [ ]:
fig1, ax1 = plt.subplots()
plt.title('Error for ' + faceApprox + ' Face Approximation RK' + str(RK) + ' on ' + grid + ' Grid')
plt.ylabel('Max Norm of Error')
plt.xlabel('Resolution')
plt.grid(True)
fig2, ax2 = plt.subplots()
plt.title('Error for ' + faceApprox + ' Face Approximation RK' + str(RK) + ' on ' + grid + ' Grid')
plt.ylabel('Max Norm of Error')
plt.xlabel('Resolution')
plt.grid(True)
plt.xscale('log')
plt.yscale('log')
for o in range(orders):
    order = 2*o + 1
    if ((deriv == 'CD') or (deriv == 'C')):
        order = order + 1
    print('Order:', order)
    print('')
    nhVec, errorTot = WaveEqConvergenceTest(omega0, physics, CFL, RK, order, diff, func, args, dataPts, AMR, numRefCel = 4, plot = plot)
    if (o == 0):
        marker = 'o'
    else:
        if (o == 1):
            marker = 's'
        else:
            marker = '^'
    print('order:', order)
    print('errorVals:')
    print(errorTot)
    slope2 = np.log(errorTot[-2] / errorTot[-1]) / np.log(nhVec[-2] / nhVec[-1])
    slope1 = np.log(errorTot[-3] / errorTot[-2]) / np.log(nhVec[-3] / nhVec[-2])
    slopeAve = (slope1 + slope2) / 2
    print('slope1 = ', slope1)
    print('slope2 = ', slope2)
    print('slope = ', slopeAve)
    ax1.plot(nhVec, errorTot)
    ax1.scatter(nhVec, errorTot, marker = marker, label = 'Order-' + str(order))
    ax2.plot(nhVec, errorTot)
    ax2.scatter(nhVec, errorTot, marker = marker, label = 'Order-' + str(order))
    
    with open(savePath + 'RK' + str(RK) + 'Order' + str(order) + grid + meds + 'LogInfo.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)

        # write the header
        writer.writerow(nhVec)
        writer.writerow(errorTot)
        writer.writerow(['', '', '', slope1, slope2])
    
# ax2.yscale('log')
refLin = nhVec.astype(float)**(-1)
ax1.plot(nhVec, refLin, color = 'k', zorder = 0)
ax2.plot(nhVec, refLin, color = 'k', zorder = 0)
ax1.legend()
ax2.legend()
fig1.savefig(saveStringLin + '.png', bbox_inches = 'tight', dpi = 300, transparent = False)
fig2.savefig(saveStringLog + '.png', bbox_inches = 'tight', dpi = 300, transparent = False)
plt.show()
plt.show()

# fig, ax = plt.subplots()
# plt.plot(nhVec, errorTot)
# plt.title('Error for Order-' + str(order) + ' ' + faceApprox + ' Face Approximation RK' + str(RK) + ' on ' + grid + ' Grid')
# plt.ylabel('Max Norm of Error')
# plt.xlabel('Resolution')
# plt.grid(True)
# plt.yscale('log')
# plt.savefig(saveStringLog + '.png', bbox_inches = 'tight', dpi = 300, transparent = True)
# plt.show()


Order: 1

[ 16  32  64 128 256]
iter: 0
nh: 16
[4, 5, 6, 7, 8, 9, 10, 11]
t: 0.02209708691207961
func is <function WaveEqRHS at 0x7f8577eeb730>
You are using WaveEqRHS()!


p: 19
q: 3
p: 19
q: 3
THIS OPERATOR IS UNIFORM!
THIS OPERATOR IS UNIFORM!
func is <function AdvectRHS at 0x7f8577eeb7b8>
You are using AdvectRHS()!
In SolverTools: AdvectRHS(), you need to fix rhsE to work in non-uniform media!
Max norm errors vs difference: 0.2686179974119234 0.2077445170566038 0.06087348035531959
Error:
[-0.000000000684423408107085 -0.00002257915368528111     -0.013924271609580558       -0.10116659039326326         0.2686179974119234          0.013810106090611285       -0.03845883834971289        -0.012607721027371604       -0.001102427097924716       -0.000031907805162035885    -0.00000032435451374894587  -0.000000001184372873965824 -0.000000000001569022826957 -0.000000000000000950628465  0.00000000000000003469447   0.00000000000000015959456   0.000000000000000058980598 -0.00000000000000015265566

func is <function WaveEqRHS at 0x7f8577eeb730>
You are using WaveEqRHS()!


p: 39
q: 7
p: 39
q: 7
THIS OPERATOR IS UNIFORM!
THIS OPERATOR IS UNIFORM!
func is <function AdvectRHS at 0x7f8577eeb7b8>
You are using AdvectRHS()!
In SolverTools: AdvectRHS(), you need to fix rhsE to work in non-uniform media!
Max norm errors vs difference: 0.09896623207271205 0.07425073545867289 0.024715496614039156
Error:
[-0.000000000001520754309853 -0.00000000116362159892699  -0.0000003235987566689168   -0.0000324015263452182      -0.0011424485848176014      -0.013382016185636187       -0.0422455850343251          0.012715599349528595        0.09896623207271205         0.017650481080814795        0.0014351958441620738      -0.009556306349752908       -0.010464557796813934       -0.006320661273986547       -0.002581035767812264       -0.0007588816304904549      -0.00016530255683121957     -0.000027092060386796086    -0.000003372457521167891    -0.00000032081711315774294  -0.000000023420890364926893 -0.00000

t: 0.005524271728019903
func is <function WaveEqRHS at 0x7f8577eeb730>
You are using WaveEqRHS()!


p: 79
q: 15
p: 79
q: 15
THIS OPERATOR IS UNIFORM!
THIS OPERATOR IS UNIFORM!
func is <function AdvectRHS at 0x7f8577eeb7b8>
You are using AdvectRHS()!
In SolverTools: AdvectRHS(), you need to fix rhsE to work in non-uniform media!
Max norm errors vs difference: 0.02768048432274217 0.02060004272313365 0.007080441599608522
Error:
[-0.000000000000049531469788 -0.000000000001961873412342 -0.000000000058862237081194 -0.000000001364791055381161 -0.00000002441393622898895  -0.000000336204475370044    -0.0000035535805294680663   -0.000028708238274054103    -0.00017618830556581384     -0.0008138155434749788      -0.0027860669389614505      -0.006873275189313743       -0.011486711081837797       -0.010676209611239273        0.0012318889678060163       0.01916337414443725         0.02768048432274217         0.005930802015777337        0.0043989152958059385       0.0022833488158471305       0.0001388

t: 0.0027621358640099515
func is <function WaveEqRHS at 0x7f8577eeb730>
You are using WaveEqRHS()!


p: 159
q: 31
p: 159
q: 31
THIS OPERATOR IS UNIFORM!
THIS OPERATOR IS UNIFORM!
func is <function AdvectRHS at 0x7f8577eeb7b8>
You are using AdvectRHS()!
In SolverTools: AdvectRHS(), you need to fix rhsE to work in non-uniform media!
Max norm errors vs difference: 0.007124752597246431 0.006521385213477338 0.0006033673837690934
Error:
[-0.000000000000004305605772 -0.000000000000041242686053 -0.000000000000263792084693 -0.000000000001553417155552 -0.000000000008555345072527 -0.00000000004414349467289  -0.000000000213341878327605 -0.000000000965460120336443 -0.000000004090085317958688 -0.000000016215565145965033 -0.00000006014148452003433  -0.0000002085806819002638   -0.0000006761082899288344   -0.0000020471340708231055   -0.000005785767398570809    -0.000015250871986804287    -0.00003745419054627377     -0.0000855899947340942      -0.00018170202824528475     -0.0003576030576844299      -0.0

t: 0.0013810679320049757
func is <function WaveEqRHS at 0x7f8577eeb730>
You are using WaveEqRHS()!


p: 319
q: 63
p: 319
q: 63
THIS OPERATOR IS UNIFORM!
THIS OPERATOR IS UNIFORM!
func is <function AdvectRHS at 0x7f8577eeb7b8>
You are using AdvectRHS()!
In SolverTools: AdvectRHS(), you need to fix rhsE to work in non-uniform media!
Max norm errors vs difference: 0.0017943369741934 0.0017553981739605318 3.893880023286833e-05
Error:
[ 0.000000000000001483648639 -0.000000000000003938699928 -0.000000000000010319215094 -0.000000000000023484318802 -0.000000000000055474948191 -0.000000000000133022932066 -0.000000000000323991494579 -0.000000000000770604966047 -0.000000000001809397726108 -0.000000000004166331961677 -0.000000000009441712214327 -0.000000000021041273438562 -0.000000000046128842364243 -0.000000000099472333309094 -0.000000000210983414814299 -0.000000000440147279940102 -0.000000000903092664848559 -0.000000001822356628727876 -0.000000003616434234722541 -0.000000007057550709414967 -0.00

...

t: 0.0027621358640099515
func is <function WaveEqRHS at 0x7f8577eeb730>
You are using WaveEqRHS()!


p: 159
q: 31
p: 159
q: 31
THIS OPERATOR IS UNIFORM!
THIS OPERATOR IS UNIFORM!
func is <function AdvectRHS at 0x7f8577eeb7b8>
You are using AdvectRHS()!
In SolverTools: AdvectRHS(), you need to fix rhsE to work in non-uniform media!
Max norm errors vs difference: 0.003749755205842753 0.0037865258056871687 -3.6770599844415663e-05
Error:
[ 0.000000000000016100768839  0.000000000000090626992265  0.000000000000533239344285  0.00000000000295905863498   0.000000000015396680689277  0.000000000075040575430662  0.000000000342511390787247  0.000000001463773690969104  0.000000005855526865741977  0.000000021918668036804335  0.00000007674696808084814   0.0000002512605869392219    0.0000007687573036421068    0.000002196857105414646     0.000005859452843513047     0.000014574168336325076     0.000033769354718206964     0.00007279397167152843      0.0001457326474780126       0.00027034807671514217      

t: 0.0013810679320049757
func is <function WaveEqRHS at 0x7f8577eeb730>
You are using WaveEqRHS()!


p: 319
q: 63
p: 319
q: 63
THIS OPERATOR IS UNIFORM!
THIS OPERATOR IS UNIFORM!
func is <function AdvectRHS at 0x7f8577eeb7b8>
You are using AdvectRHS()!
In SolverTools: AdvectRHS(), you need to fix rhsE to work in non-uniform media!
Max norm errors vs difference: 0.0009719930090636986 0.0009742979157512099 -2.3049066875113766e-06
Error:
[ 0.000000000000004041589089  0.00000000000000230195857   0.000000000000006537224656  0.0000000000000197573355    0.000000000000051269981253  0.000000000000128130294574  0.000000000000303439671647  0.000000000000713798947192  0.000000000001643784696105  0.000000000003738521693167  0.000000000008359316348464  0.000000000018393757700086  0.000000000039809865689615  0.000000000084756134550239  0.000000000177495636510332  0.000000000365614606234761  0.000000000740738837379259  0.000000001476028571579694  0.000000002892648054830632  0.000000005575020026325296 

func is <function AdvectRHS at 0x7f8577eeb7b8>
You are using AdvectRHS()!
In SolverTools: AdvectRHS(), you need to fix rhsE to work in non-uniform media!
Max norm errors vs difference: 0.06631387877364611 0.06631387877364617 -5.551115123125783e-17
Error:
[-0.00048625416479019856     -0.005646821816987398        0.07439807850832642        -0.1368017959871503          0.011676860860495686        0.034113825292522826        0.06631387877364611         0.025557091425397332        0.0001870306592514537      -0.0007099712631774926      -0.00012769546063807902      0.0002917123767714054      -0.00028300802322348706      0.00007784246083578084     -0.00002266685679149817      0.000000000000000246330734  0.000000000000000055511151 -0.000000000000000180411242 -0.000000000000000235922393 -0.000000000000000006938894  0.000000000000000235922393 -0.000000000000000029490299 -0.000000000000000058546917 -0.000000000000000013010426 -0.00048625416479019856     -0.005646821816987398        0.0743980785083

func is <function WaveEqRHS at 0x7f8577eeb730>
You are using WaveEqRHS()!


p: 39
q: 7
p: 39
q: 7
THIS OPERATOR IS UNIFORM!
THIS OPERATOR IS UNIFORM!
func is <function AdvectRHS at 0x7f8577eeb7b8>
You are using AdvectRHS()!
In SolverTools: AdvectRHS(), you need to fix rhsE to work in non-uniform media!
Max norm errors vs difference: 0.03901662892512214 0.06233965077795889 -0.023323021852836745
Error:
[-0.000000011084993811810119 -0.0000011892490008935133   -0.00004361018389046845     -0.0004176934940589509       0.001425548682020228        0.01890400567298502         0.014308036719487854       -0.03901662892512214        -0.0346638388146453         -0.02831067963860412         0.006757837773695319        0.0253493063615699          0.02260700919618394         0.012200553466677186        0.0044575531754349855       0.0011089024897545188       0.00017175205598338704      0.000008807589912619186    -0.000003007325597177929    -0.0000009458253214617626   -0.0000001495274098990379   -0.0000

t: 0.005524271728019903


In [ ]:
print(refLin)
print('')

In [ ]:
# def ConvergTest1(omega, deriv, order):
#     # Write in error trap for wavenumber and nh_max.
#     nh_max = omega.nh_max
#     iters = int(np.log(nh_max) / np.log(2))
#     wave = WT.MakeWaves(omega)
#     testWave = 2 * np.pi * wave[:, 2]
#     waves = wave[:, 1]
#     DerivOp = 2 * OT.SpaceDeriv(omega, order, deriv)
#     CoarseOp = GTT.MakeFullWeight(nh_max)
#     exactDerivOp = TT.ExactSpatOp(omega) # , 0, waves, -1, order)
#     exactDeriv = waves @ exactDerivOp
#     nhs = 2 ** (np.arange(iters) + 1)
#     print(iters)
#     NormVec = []#[[] for i in range()]
#     for i in range(iters):
#         print(i)
#         nh = int(nh_max / (2 ** i))
#         DerivOp = 0.5 * DerivOp[:nh, :nh]
#         DerivOp[0, nh - 1] = DerivOp[1, 0]
#         DerivOp[nh - 1, 0] = DerivOp[0, 1]
#         if (i > 0):
#             CoarseOp = CoarseOp[:nh, :int(2 * nh)]
#             waves = CoarseOp @ waves
#             testWave = CoarseOp @ testWave
#             exactDeriv = CoarseOp @ exactDeriv
#         testDeriv = DerivOp @ waves#[:, :nh]
#         errorVec = abs(testDeriv - exactDeriv)#[:, :nh])
#         NormVec.append(TT.VectorNorm(errorVec))
#         print('test and exact:')
#         print(testDeriv)
#         print(exactDeriv)
#         print(testWave)
#         print(errorVec)
#         print(errorVec ** 2)
#         print(sum(errorVec ** 2))
#         print(np.sqrt(sum(errorVec ** 2)))
#         print('')
#         print(NormVec[i])
#     NormVec = np.asarray(NormVec)[::-1].transpose()
#     print('NormVec is')
#     print(NormVec)
#     return nhs, NormVec

In [ ]:
# def ConvergTest(omega, deriv, order):
#     # Write in error trap for wavenumber and nh_max.
#     nh_max = omega.nh_max
#     iters = int(np.log(nh_max) / np.log(2))
#     waves = WT.MakeWaves(omega)
# #     testWave = 2 * np.pi * wave[:, 2]
# #     waves = wave[:, 1]
#     DerivOp = 2 * OT.SpaceDeriv(omega, order, deriv)
#     CoarseOp = GTT.MakeFullWeight(nh_max)
#     exactDerivOp = TT.ExactSpatOp(omega) #, 0, waves, -1, order)
#     exactDeriv = waves @ exactDerivOp
#     nhs = 2 ** (np.arange(iters) + 1)
#     print(iters)
#     NormVecs = np.zeros((iters, nh_max), float)#[[] for i in range()]
#     mask = np.zeros(nh_max, int)
#     masks = []
#     for i in range(iters):
#         print(i)
#         nh = int(nh_max / (2 ** i))
        
#         mask[:nh] = 1
#         masks.append(list(mask))
#         # mask = list(mask)
#         print('mask:')
#         print(mask)
#         print(masks)
#         DerivOp = 0.5 * DerivOp[:nh, :nh]
#         DerivOp[0, nh - 1] = DerivOp[1, 0]
#         DerivOp[nh - 1, 0] = DerivOp[0, 1]
#         if (i > 0):
#             CoarseOp = CoarseOp[:nh, :int(2 * nh)]
#             waves = CoarseOp @ waves
# #             testWave = CoarseOp @ testWave
#             exactDeriv = CoarseOp @ exactDeriv
#         testDeriv = DerivOp @ waves#[:, :nh]
#         errorVec = abs(testDeriv - exactDeriv)#[:, :nh])
#         NormVec = TT.VectorNorm(errorVec)
        
#         #NormVec = ma.masked_array(NormVec, mask = mask)
        
#         NormVecs[i, :] = NormVec
#         print('Before Mask:')
#         print(NormVecs[i, :])
        
#         print('After Mask:')
#         print(NormVecs[i, :])
#         #print(NormVecs[::-1])
#         print('test and exact:')
#         print(testDeriv)
#         print(exactDeriv)
# #         print(testWave)
#         print(errorVec)
#         print(errorVec ** 2)
#         print(sum(errorVec ** 2, axis = 0))
#         print(np.sqrt(sum(errorVec ** 2, axis = 0)))
#         print('')
#         print(NormVecs[i])
#     masks = [item for sublist in masks for item in sublist]
#     print(masks)
#     print(NormVecs)
#     NormVecs = np.asarray(NormVecs)[::-1]#.transpose()
#     ma.masked_array(NormVecs, mask = list(masks))
#     print('NormVecs is')
#     print(NormVecs)
#     return nhs, NormVecs

In [ ]:
# nhs1, NormVec1 = ConvergTest1(omega, deriv, order)
# print(nhs1)


In [ ]:
# nhs, NormVec = ConvergTest(omega, deriv, order)
# print(nhs)


In [ ]:
# plt.figure()
# plt.plot(nhs1, NormVec1)
# print(NormVec1)
# print('')
# plt.show()

In [ ]:
# plt.figure()
# plt.plot(nhs, NormVec[:, 1])
# print(NormVec[:, 1])
# print('')
# plt.yscale('log')
# plt.show()

In [ ]:
# print('32:', [0.16092122, 0.45034975, 1.22071342, 2.90704182, 0.90031632])
# print('512:', [2.52326242e-03, 7.13656831e-03, 2.01819198e-02, 5.70452747e-02, 1.60921219e-01, 4.50349746e-01, 1.22071342e+00, 2.90704182e+00, 9.00316316e-01])

In [ ]:
# A = np.arange(16).reshape(4, 4)
# print(A)
# print(A ** 2)
# print('')
# print(sum(A, axis = 0))
# out = TT.VectorNorm(A)
# print(out)
# print(A[:, :2])
# A = ma.masked_array(A, mask=[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])
# print(A)
# print(type([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1]))
# C = np.ones(6, float)
# print(type(C))
# C = list(C)
# print(type(C))

In [ ]:
nh=32
halfO=4
nt=1


In [ ]:



omega = BT.Grid(nh)
finepart = list(int(nh / 2) + np.arange(-int((halfO * nh) / nh_min), int((halfO * nh) / nh_min)))
# omega.AddPatch(refRatio, finepart)
print(finepart)

xCell = omega.xCell
physics = BT.PhysProps(omega, epsilons, mus, locs, L)
degFreed = omega.degFreed
waves = WT.MakeWaves(omega)
nullspace = OT.FindNullspace(omega, waves)
#         waves = waves @ nullspace
wavesMat = OT.Block(waves, var = 2)
nullspaceBlock = OT.Block(nullspace, var = 2)

c = physics.cVec
cMat = physics.cMat
c1 = c[0]

t, nt = ST.CalcTime(omega, CFL, cMat, nt = nt, override = True)


waveInit = WFT.InitCond(omega, physics, func, args)
FCoefs = FFTT.FourierCoefs(waves @ nullspace, waveInit)
print('t:', t)

wavePropEx = WFT.WaveEq(omega, physics, func, args, t)

# Find propagated coeficients on coarse and fine grid.
FCoefsProp = FFTT.FourierCoefs(waves @ nullspace, wavePropEx)

solverCoefs = ST.RungeKutta(omega, physics, waveInit, CFL, nt, RK, order, deriv, ST.WaveEqRHS, override = True)
wavePropSolv = wavesMat @ nullspaceBlock @ solverCoefs

errorVec = wavePropEx - wavePropSolv
                
errorCoefs = PT.Load(FFTT.FourierCoefs(waves @ nullspace, errorVec))

PT.PlotMixedWave(omega, physics, waves = waves, FCoefs = nullspace @ errorCoefs[:degFreed], rescale = 5, yGrid = True, plotCont = False, title = 'Error on E Field')

allCoefs = PT.Load(FCoefs, FCoefsProp, solverCoefs)
allCoefs = nullspaceBlock @ allCoefs

labels = ['Initial Wave', 'Exact Propagated Wave', 'RK-Propagated Wave']


errorCoefs = PT.Load(FFTT.FourierCoefs(waves @ nullspace, errorVec))

PT.PlotMixedWave(omega, physics, waves = waves, FCoefs = allCoefs, rescale = 5, yGrid = True, labels = labels, plotCont = False)


In [ ]:
omega = BT.Grid(nh)
finepart = list(int(nh / 2) + np.arange(-int((halfO * nh) / nh_min), int((halfO * nh) / nh_min)))
omega.AddPatch(refRatio, finepart)
print(finepart)

xCell = omega.xCell
physics = BT.PhysProps(omega, epsilons, mus, locs, L)
degFreed = omega.degFreed
waves = WT.MakeWaves(omega)
nullspace = OT.FindNullspace(omega, waves)
#         waves = waves @ nullspace
wavesMat = OT.Block(waves, var = 2)
nullspaceBlock = OT.Block(nullspace, var = 2)

c = physics.cVec
cMat = physics.cMat
c1 = c[0]

t, nt = ST.CalcTime(omega, CFL, cMat, nt = nt, override = True)


waveInit = WFT.InitCond(omega, physics, func, args)
FCoefs = FFTT.FourierCoefs(waves @ nullspace, waveInit)
print('t:', t)

wavePropEx = WFT.WaveEq(omega, physics, func, args, t)

# Find propagated coeficients on coarse and fine grid.
FCoefsProp = FFTT.FourierCoefs(waves @ nullspace, wavePropEx)

solverCoefs = ST.RungeKutta(omega, physics, waveInit, CFL, nt, RK, order, deriv, ST.WaveEqRHS, override = True)
wavePropSolv = wavesMat @ nullspaceBlock @ solverCoefs

errorVec = wavePropEx - wavePropSolv

errorCoefs = PT.Load(FFTT.FourierCoefs(waves @ nullspace, errorVec))

PT.PlotMixedWave(omega, physics, waves = waves, FCoefs = nullspace @ errorCoefs[:degFreed], rescale = 5, yGrid = True, plotCont = False, title = 'Error on E Field')


allCoefs = PT.Load(FCoefs, FCoefsProp, solverCoefs)
allCoefs = nullspaceBlock @ allCoefs

labels = ['Initial Wave', 'Exact Propagated Wave', 'RK-Propagated Wave']


errorCoefs = PT.Load(FFTT.FourierCoefs(waves @ nullspace, errorVec))

PT.PlotMixedWave(omega, physics, waves = waves, FCoefs = allCoefs, rescale = 5, yGrid = True, labels = labels, plotCont = False)


In [ ]:

nh = nh*2

omega = BT.Grid(nh)
finepart = list(int(nh / 2) + np.arange(-int((halfO * nh) / nh_min), int((halfO * nh) / nh_min)))
# omega.AddPatch(refRatio, finepart)
print(finepart)

xCell = omega.xCell
physics = BT.PhysProps(omega, epsilons, mus, locs, L)
degFreed = omega.degFreed
waves = WT.MakeWaves(omega)
nullspace = OT.FindNullspace(omega, waves)
#         waves = waves @ nullspace
wavesMat = OT.Block(waves, var = 2)
nullspaceBlock = OT.Block(nullspace, var = 2)

c = physics.cVec
cMat = physics.cMat
c1 = c[0]

t, nt = ST.CalcTime(omega, CFL, cMat, nt = nt, override = True)


waveInit = WFT.InitCond(omega, physics, func, args)
FCoefs = FFTT.FourierCoefs(waves @ nullspace, waveInit)
print('t:', t)

wavePropEx = WFT.WaveEq(omega, physics, func, args, t)

# Find propagated coeficients on coarse and fine grid.
FCoefsProp = FFTT.FourierCoefs(waves @ nullspace, wavePropEx)

solverCoefs = ST.RungeKutta(omega, physics, waveInit, CFL, nt, RK, order, deriv, ST.WaveEqRHS, override = True)
wavePropSolv = wavesMat @ nullspaceBlock @ solverCoefs

errorVec = wavePropEx - wavePropSolv
                
errorCoefs = PT.Load(FFTT.FourierCoefs(waves @ nullspace, errorVec))

PT.PlotMixedWave(omega, physics, waves = waves, FCoefs = nullspace @ errorCoefs[:degFreed], rescale = 5, yGrid = True, plotCont = False, title = 'Error on E Field')

allCoefs = PT.Load(FCoefs, FCoefsProp, solverCoefs)
allCoefs = nullspaceBlock @ allCoefs

labels = ['Initial Wave', 'Exact Propagated Wave', 'RK-Propagated Wave']


errorCoefs = PT.Load(FFTT.FourierCoefs(waves @ nullspace, errorVec))

PT.PlotMixedWave(omega, physics, waves = waves, FCoefs = allCoefs, rescale = 5, yGrid = True, labels = labels, plotCont = False)


In [ ]:
A = np.arange(64).reshape(8, 8)
print(A)
print(A[1:, 2:-2])
print('')